In [30]:
# Written with help from the Full Control documentation Jupyter notebooks
# Created by Hannah Eikens for Final Bachelor Project @ TU/e, department of Industrial Design

In [31]:
import fullcontrol as fc
from fabuloushelpers import single_line_quill_rachisXY

In [32]:
# printer/gcode parameters

design_name = 'fabulous_quills'
nozzle_diameter = 0.4
nozzle_temp = 220 # PET 220
bed_temp = 65 # PET 65
print_speed = 500
fan_percent = 0
fan_percent_overhang = 50 # fan speed on overhangs
retraction = 3.0 # set to 0.0 to disable
z_lift = 0.6
z_offset = -0.065 # MANUALLY ADJUST Z OFFSET adjust z-offset to get good first layer
material_flow_percent = 99
printer_name='ender_3' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [33]:
# design parameters

EW = 1.0*nozzle_diameter # extrusion width
EH = 0.2 # extrusion height (and layer height)

start_point = fc.Point(x=20, y=15, z=0)

rachis_length = 50
quill_length = 15 # mm, length of the end of the shaft
quill_width = 2.0 # mm, width of the quill
quill_EH = 0.2 # mm
quill_height = 4*quill_EH # mm height of the quill
quill_speed = 100
afterfeather_length = 10

wipe_distance = quill_length/2

# batch settings
x_quills = 2
y_quills = 16
feather_spacing = 10

In [34]:
# generate the design (make sure you've run the above cells before running this cell)

steps = []

rachis_steps = []
rachis_steps.append(fc.ExtrusionGeometry(height=quill_EH))
rachis_layers = round(quill_height/quill_EH)
for  layer in range(rachis_layers):
    rachis_layer_steps = []
    # travel to begin of rachis
    rachis_steps.extend(fc.travel_to(fc.Point(x=quill_length+afterfeather_length+(rachis_length+0.25-rachis_length/4*layer), 
                                        y=0, 
                                        z=(quill_EH+layer*quill_EH)+z_lift
                                        )
                                    )
                        )
    # draw rachis layer
    rachis_layer_steps.extend(single_line_quill_rachisXY(fc.Point(x=0, y=0, z=EH+layer*quill_EH), 
                                                            quill_length=quill_length+afterfeather_length, 
                                                            quill_width=quill_width-quill_width/10*layer, 
                                                            rachis_length=rachis_length+0.25-rachis_length/4*layer ,
                                                            max_extrusion_width=quill_width,
                                                            segments=int(rachis_length*4)
                                                            )
                              )
    # wipe nozzle
    rachis_layer_steps.extend(fc.travel_to(fc.Point(x=rachis_layer_steps[-1].x+wipe_distance, 
                                                y=rachis_layer_steps[-1].y, 
                                                z=rachis_layer_steps[-1].z
                                                )
                                        )
                                )
    rachis_layer_steps.extend(fc.travel_to(fc.Point(x=rachis_layer_steps[-2].x-wipe_distance, 
                                                y=rachis_layer_steps[-2].y, 
                                                z=rachis_layer_steps[-2].z
                                                )
                                        )
                                )

    # lift z
    rachis_layer_steps.extend(fc.travel_to(fc.Point(x=rachis_layer_steps[-2].x, 
                                                y=rachis_layer_steps[-2].y, 
                                                z=rachis_layer_steps[-2].z+z_lift
                                                )
                                        )
                                )
    rachis_steps.extend(rachis_layer_steps)
    

steps.append(fc.Printer(print_speed=quill_speed))
steps.extend(rachis_steps)

steps = fc.move(steps, fc.Vector(x=start_point.x, y=start_point.y))

steps = fc.move(steps, fc.Vector(x=rachis_length+afterfeather_length+quill_length+feather_spacing), 
                copy=True, 
                copy_quantity=x_quills
                ) # copy x
steps = fc.move(steps, fc.Vector(y=quill_width+feather_spacing), 
                copy=True, 
                copy_quantity=y_quills
                ) # copy y

In [35]:
# preview the design

fc.transform(steps, 'plot')
# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
# fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True)) 

In [36]:
# create and download gcode for the design

gcode_controls = fc.GcodeControls(
    printer_name = printer_name, 
    save_as = design_name + "_EW" + str(EW) + "_EH" + str(EH),
    initialization_data={
    'primer': 'front_lines_then_x', 
    'print_speed': print_speed,
    "nozzle_temp": nozzle_temp,
    "bed_temp": bed_temp,
    'fan_percent': fan_percent,
    "extrusion_width": EW,
    "extrusion_height": EH,
    "material_flow_percent": material_flow_percent})
gcode = fc.transform(steps, 'gcode', gcode_controls)